# PyCitySchools Challenge

## Report

Analysis was performed on the data set to determine school performance on several metrics, including school spending per student, school size, and school type. School performance was determined by averaging math and reading scores, using the averages to calculate based on a passing score of 70, and then determing the overall passing rate as a combination of math and reading passing rates.

School spending was categorized by the following spending ranges per student:

    - <$585
    - $585-630
    - $630-645
    - $645-680

The best performing schools spent the least amount of money per student as measured by % Overall Passing Rate (90.4% Overall passing). There's a direct negative correlation between spend and passing rate as passing rate decreases as per student spend increases. The bottom performing schools (overall passing rate of 53.5%) fell in the highest spending per student bracket.

Schools of small (<1000 students) and medium (1000-2000 students) size performed similarly well as measured by % Overall passing rate (89.9% and 90.6% respectively). Large schools (2000-5000 students) significantly underperformed with a passing rate of 58.3%.

Charter schools significantly outperformed district schools with an overall passing rate of 90.4% vs 53.7% respectively. 



In [121]:
import pandas as pd
from pathlib import Path

schools_csv = "../Resources/schools_complete.csv"
students_csv = "../Resources/students_complete.csv"

schools_df = pd.read_csv(schools_csv)
students_df = pd.read_csv(students_csv)

district_df = pd.merge(students_df, schools_df, on="school_name", how="left")
district_df

,Student ID,student_name,gender,grade,school_name,reading_score,math_score,School ID,type,size,budget
0,0,Paul Bradley,M,9th,Huang High School,66,79,0,District,2917,1910635
1,1,Victor Smith,M,12th,Huang High School,94,61,0,District,2917,1910635
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,0,District,2917,1910635
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58,0,District,2917,1910635
4,4,Bonnie Ray,F,9th,Huang High School,97,84,0,District,2917,1910635
...,...,...,...,...,...,...,...,...,...,...,...
39165,39165,Donna Howard,F,12th,Thomas High School,99,90,14,Charter,1635,1043130
39166,39166,Dawn Bell,F,10th,Thomas High School,95,70,14,Charter,1635,1043130
39167,39167,Rebecca Tanner,F,9th,Thomas High School,73,84,14,Charter,1635,1043130
39168,39168,Desiree Kidd,F,10th,Thomas High School,99,90,14,Charter,1635,1043130


## District Summary

In [122]:
unique_schools = len(district_df["school_name"].unique())
unique_schools

15

In [123]:
student_count = len(district_df["student_name"])
student_count

39170

In [124]:
total_budget = schools_df["budget"].sum()
total_budget

24649428

In [125]:
avg_math_score = district_df["math_score"].mean()
avg_math_score

78.98537145774827

In [126]:
avg_reading_score = district_df["reading_score"].mean()
avg_reading_score

81.87784018381414

In [127]:
count_passed_math = district_df[(district_df["math_score"] >= 70)].count()["student_name"]
percent_passed_math = count_passed_math / student_count * 100
percent_passed_math

74.9808526933878

In [128]:
count_passed_reading = district_df[(district_df["reading_score"] >= 70)].count()["student_name"]
percent_passed_reading = count_passed_reading / student_count * 100
percent_passed_reading

85.80546336482001

In [129]:
count_passed_all = district_df[(district_df["math_score"] >= 70) & (district_df["reading_score"] >= 70)].count()["student_name"]
overall_passed_percent = count_passed_all / student_count * 100
overall_passed_percent

65.17232575950983

In [130]:
district_summary = [{"Total Schools": unique_schools, "Total Students": student_count, "Total Budget": total_budget, 
                    "Average Math Score": avg_math_score, "Average Reading Score": avg_reading_score, "% Passing Math": percent_passed_math,
                    "% Passing Reading": percent_passed_reading, "% Overall Passing": overall_passed_percent}]
district_summary_df = pd.DataFrame(district_summary)

district_summary_df["Total Students"] = district_summary_df["Total Students"].map("{:,}".format)
district_summary_df["Total Budget"] = district_summary_df["Total Budget"].map("${:,.2f}".format)

district_summary_df

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
0,15,"39,170","$24,649,428.00",78.985371,81.87784,74.980853,85.805463,65.172326


## School Summary

In [131]:
schools_grouped_df = district_df.groupby(["school_name"])


I worked with Thomas Tellner and Kenneth Leung during office hours for help on how to pull district type by school into a dataframe as a string. During my previous attempts I was pulling the school type from the schools_grouped_df dataframe in the previous code cell and the type values were being stored as arrays in school_types.

In [132]:
school_types = district_df[["school_name", "type"]].drop_duplicates().sort_values(by=["school_name"]).set_index(["school_name"])  
school_types

,type
school_name,
Bailey High School,District
Cabrera High School,Charter
Figueroa High School,District
Ford High School,District
Griffin High School,Charter
Hernandez High School,District
Holden High School,Charter
Huang High School,District
Johnson High School,District


In [133]:
per_school_counts = schools_grouped_df["student_name"].count()
per_school_counts

school_name
Bailey High School       4976
Cabrera High School      1858
Figueroa High School     2949
Ford High School         2739
Griffin High School      1468
Hernandez High School    4635
Holden High School        427
Huang High School        2917
Johnson High School      4761
Pena High School          962
Rodriguez High School    3999
Shelton High School      1761
Thomas High School       1635
Wilson High School       2283
Wright High School       1800
Name: student_name, dtype: int64

In [134]:
per_school_budget = schools_grouped_df["budget"].sum() / per_school_counts
per_school_budget

school_name
Bailey High School       3124928.0
Cabrera High School      1081356.0
Figueroa High School     1884411.0
Ford High School         1763916.0
Griffin High School       917500.0
Hernandez High School    3022020.0
Holden High School        248087.0
Huang High School        1910635.0
Johnson High School      3094650.0
Pena High School          585858.0
Rodriguez High School    2547363.0
Shelton High School      1056600.0
Thomas High School       1043130.0
Wilson High School       1319574.0
Wright High School       1049400.0
dtype: float64

In [135]:
per_school_capita = per_school_budget / per_school_counts
per_school_capita

school_name
Bailey High School       628.0
Cabrera High School      582.0
Figueroa High School     639.0
Ford High School         644.0
Griffin High School      625.0
Hernandez High School    652.0
Holden High School       581.0
Huang High School        655.0
Johnson High School      650.0
Pena High School         609.0
Rodriguez High School    637.0
Shelton High School      600.0
Thomas High School       638.0
Wilson High School       578.0
Wright High School       583.0
dtype: float64

In [136]:
per_school_math = schools_grouped_df["math_score"].sum() / per_school_counts
per_school_math

school_name
Bailey High School       77.048432
Cabrera High School      83.061895
Figueroa High School     76.711767
Ford High School         77.102592
Griffin High School      83.351499
Hernandez High School    77.289752
Holden High School       83.803279
Huang High School        76.629414
Johnson High School      77.072464
Pena High School         83.839917
Rodriguez High School    76.842711
Shelton High School      83.359455
Thomas High School       83.418349
Wilson High School       83.274201
Wright High School       83.682222
dtype: float64

In [137]:
per_school_reading = schools_grouped_df["reading_score"].sum() / per_school_counts
per_school_reading

school_name
Bailey High School       81.033963
Cabrera High School      83.975780
Figueroa High School     81.158020
Ford High School         80.746258
Griffin High School      83.816757
Hernandez High School    80.934412
Holden High School       83.814988
Huang High School        81.182722
Johnson High School      80.966394
Pena High School         84.044699
Rodriguez High School    80.744686
Shelton High School      83.725724
Thomas High School       83.848930
Wilson High School       83.989488
Wright High School       83.955000
dtype: float64

In [138]:
students_passing_math = district_df[(district_df["math_score"] >= 70)]
students_passing_math = students_passing_math.groupby(["school_name"]).size()
students_passing_math_percent = students_passing_math / per_school_counts * 100
students_passing_math_percent

school_name
Bailey High School       66.680064
Cabrera High School      94.133477
Figueroa High School     65.988471
Ford High School         68.309602
Griffin High School      93.392371
Hernandez High School    66.752967
Holden High School       92.505855
Huang High School        65.683922
Johnson High School      66.057551
Pena High School         94.594595
Rodriguez High School    66.366592
Shelton High School      93.867121
Thomas High School       93.272171
Wilson High School       93.867718
Wright High School       93.333333
dtype: float64

In [139]:
students_passing_reading = district_df[(district_df["reading_score"] >= 70)]
students_passing_reading = students_passing_reading.groupby(["school_name"]).size()
students_passing_reading_percent = students_passing_reading / per_school_counts * 100
students_passing_reading_percent

school_name
Bailey High School       81.933280
Cabrera High School      97.039828
Figueroa High School     80.739234
Ford High School         79.299014
Griffin High School      97.138965
Hernandez High School    80.862999
Holden High School       96.252927
Huang High School        81.316421
Johnson High School      81.222432
Pena High School         95.945946
Rodriguez High School    80.220055
Shelton High School      95.854628
Thomas High School       97.308869
Wilson High School       96.539641
Wright High School       96.611111
dtype: float64

In [140]:
students_passing_both = district_df[(district_df["math_score"] >= 70) & (district_df["reading_score"] >= 70)]
students_passing_both = students_passing_both.groupby(["school_name"]).size()
students_passing_both_percent = students_passing_both / per_school_counts * 100
students_passing_both_percent

school_name
Bailey High School       54.642283
Cabrera High School      91.334769
Figueroa High School     53.204476
Ford High School         54.289887
Griffin High School      90.599455
Hernandez High School    53.527508
Holden High School       89.227166
Huang High School        53.513884
Johnson High School      53.539172
Pena High School         90.540541
Rodriguez High School    52.988247
Shelton High School      89.892107
Thomas High School       90.948012
Wilson High School       90.582567
Wright High School       90.333333
dtype: float64

In [141]:
per_school_summary = pd.DataFrame({"School Type": school_types["type"], "Total Students": per_school_counts, 
                        "Total School Budget": per_school_budget, "Per Student Budget": per_school_capita,
                        "Average Math Score": per_school_math, "Average Reading Score": per_school_reading,
                        "% Passing Math": students_passing_math_percent, "% Passing Reading": students_passing_reading_percent,
                        "% Overall Passing": students_passing_both_percent})


per_school_summary["Total School Budget"] = per_school_summary["Total School Budget"].map("${:,.2f}".format)
per_school_summary["Per Student Budget"] =  per_school_summary["Per Student Budget"].map("${:,.2f}".format)

per_school_summary.index.name = None # from the starter code that was provided

per_school_summary

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Bailey High School,District,4976,"$3,124,928.00",$628.00,77.048432,81.033963,66.680064,81.933280,54.642283
Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.061895,83.975780,94.133477,97.039828,91.334769
Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.711767,81.158020,65.988471,80.739234,53.204476
Ford High School,District,2739,"$1,763,916.00",$644.00,77.102592,80.746258,68.309602,79.299014,54.289887
Griffin High School,Charter,1468,"$917,500.00",$625.00,83.351499,83.816757,93.392371,97.138965,90.599455
Hernandez High School,District,4635,"$3,022,020.00",$652.00,77.289752,80.934412,66.752967,80.862999,53.527508
Holden High School,Charter,427,"$248,087.00",$581.00,83.803279,83.814988,92.505855,96.252927,89.227166
Huang High School,District,2917,"$1,910,635.00",$655.00,76.629414,81.182722,65.683922,81.316421,53.513884
Johnson High School,District,4761,"$3,094,650.00",$650.00,77.072464,80.966394,66.057551,81.222432,53.539172
Pena High School,Charter,962,"$585,858.00",$609.00,83.839917,84.044699,94.594595,95.945946,90.540541


## Highest-Performing Schools (by % Overall Passing)

In [142]:
top_schools = per_school_summary.sort_values("% Overall Passing", ascending=False)

top_schools.index.name = None # from the starter code that was provided

top_schools.head(5)

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.061895,83.975780,94.133477,97.039828,91.334769
Thomas High School,Charter,1635,"$1,043,130.00",$638.00,83.418349,83.848930,93.272171,97.308869,90.948012
Griffin High School,Charter,1468,"$917,500.00",$625.00,83.351499,83.816757,93.392371,97.138965,90.599455
Wilson High School,Charter,2283,"$1,319,574.00",$578.00,83.274201,83.989488,93.867718,96.539641,90.582567
Pena High School,Charter,962,"$585,858.00",$609.00,83.839917,84.044699,94.594595,95.945946,90.540541


## Bottom-Performing Schools (by % Overall Passing)

In [143]:
bottom_schools = per_school_summary.sort_values("% Overall Passing")

bottom_schools.index.name = None # from the starter code that was provided

bottom_schools.head(5)

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Rodriguez High School,District,3999,"$2,547,363.00",$637.00,76.842711,80.744686,66.366592,80.220055,52.988247
Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.711767,81.158020,65.988471,80.739234,53.204476
Huang High School,District,2917,"$1,910,635.00",$655.00,76.629414,81.182722,65.683922,81.316421,53.513884
Hernandez High School,District,4635,"$3,022,020.00",$652.00,77.289752,80.934412,66.752967,80.862999,53.527508
Johnson High School,District,4761,"$3,094,650.00",$650.00,77.072464,80.966394,66.057551,81.222432,53.539172


## Math Scores by Grade

In [144]:
ninth_graders = district_df[(district_df["grade"] == "9th")]
tenth_graders = district_df[(district_df["grade"] == "10th")]
eleventh_graders = district_df[(district_df["grade"] == "11th")]
twelfth_graders = district_df[(district_df["grade"] == "12th")]

ninth_grade_math_scores = ninth_graders.groupby(["school_name"]).mean("math_score")
tenth_grade_math_scores = tenth_graders.groupby(["school_name"]).mean("math_score")
eleventh_grade_math_scores = eleventh_graders.groupby(["school_name"]).mean("math_score")
twelfth_grade_math_scores = twelfth_graders.groupby(["school_name"]).mean("math_score")

math_scores_by_grade = pd.DataFrame({"9th": ninth_grade_math_scores["math_score"], "10th": tenth_grade_math_scores["math_score"],
                                    "11th": eleventh_grade_math_scores["math_score"], "12th": twelfth_grade_math_scores["math_score"]})

math_scores_by_grade.index.name = None

math_scores_by_grade["9th"] = math_scores_by_grade["9th"].map("{:.2f}".format)
math_scores_by_grade["10th"] = math_scores_by_grade["10th"].map("{:.2f}".format)
math_scores_by_grade["11th"] = math_scores_by_grade["11th"].map("{:.2f}".format)
math_scores_by_grade["12th"] = math_scores_by_grade["12th"].map("{:.2f}".format)

math_scores_by_grade

,9th,10th,11th,12th
Bailey High School,77.08,77.00,77.52,76.49
Cabrera High School,83.09,83.15,82.77,83.28
Figueroa High School,76.40,76.54,76.88,77.15
Ford High School,77.36,77.67,76.92,76.18
Griffin High School,82.04,84.23,83.84,83.36
Hernandez High School,77.44,77.34,77.14,77.19
Holden High School,83.79,83.43,85.00,82.86
Huang High School,77.03,75.91,76.45,77.23
Johnson High School,77.19,76.69,77.49,76.86
Pena High School,83.63,83.37,84.33,84.12


## Reading Scores by Grade

In [145]:
ninth_grade_reading_scores = ninth_graders.groupby(["school_name"]).mean(["reading_score"])
tenth_grade_reading_scores = tenth_graders.groupby(["school_name"]).mean(["reading_score"])
eleventh_grade_reading_scores = eleventh_graders.groupby(["school_name"]).mean(["reading_score"])
twelfth_grade_reading_scores = twelfth_graders.groupby(["school_name"]).mean(["reading_score"])

reading_scores_by_grade = pd.DataFrame({"9th": ninth_grade_reading_scores["reading_score"], 
                                        "10th": tenth_grade_reading_scores["reading_score"],
                                        "11th": eleventh_grade_reading_scores["reading_score"], 
                                        "12th": twelfth_grade_reading_scores["reading_score"]})

reading_scores_by_grade["9th"] = reading_scores_by_grade["9th"].map("{:.2f}".format)
reading_scores_by_grade["10th"] = reading_scores_by_grade["10th"].map("{:.2f}".format)
reading_scores_by_grade["11th"] = reading_scores_by_grade["11th"].map("{:.2f}".format)
reading_scores_by_grade["12th"] = reading_scores_by_grade["12th"].map("{:.2f}".format)

reading_scores_by_grade.index.name = None

reading_scores_by_grade

,9th,10th,11th,12th
Bailey High School,81.30,80.91,80.95,80.91
Cabrera High School,83.68,84.25,83.79,84.29
Figueroa High School,81.20,81.41,80.64,81.38
Ford High School,80.63,81.26,80.40,80.66
Griffin High School,83.37,83.71,84.29,84.01
Hernandez High School,80.87,80.66,81.40,80.86
Holden High School,83.68,83.32,83.82,84.70
Huang High School,81.29,81.51,81.42,80.31
Johnson High School,81.26,80.77,80.62,81.23
Pena High School,83.81,83.61,84.34,84.59


## Schools by School Spending

In [146]:
spending_bins = [0, 585, 630, 645, 680]

spending_labels = ["<$585", "$585-630", "$630-645", "$645-680"]

school_spending_df = per_school_summary.copy() # from the starter code that was provided

school_spending_df["Spending Ranges (Per Student)"] = pd.cut(per_school_capita, spending_bins, labels=spending_labels, include_lowest=True)
school_spending_df

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing,Spending Ranges (Per Student)
Bailey High School,District,4976,"$3,124,928.00",$628.00,77.048432,81.033963,66.680064,81.933280,54.642283,$585-630
Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.061895,83.975780,94.133477,97.039828,91.334769,<$585
Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.711767,81.158020,65.988471,80.739234,53.204476,$630-645
Ford High School,District,2739,"$1,763,916.00",$644.00,77.102592,80.746258,68.309602,79.299014,54.289887,$630-645
Griffin High School,Charter,1468,"$917,500.00",$625.00,83.351499,83.816757,93.392371,97.138965,90.599455,$585-630
Hernandez High School,District,4635,"$3,022,020.00",$652.00,77.289752,80.934412,66.752967,80.862999,53.527508,$645-680
Holden High School,Charter,427,"$248,087.00",$581.00,83.803279,83.814988,92.505855,96.252927,89.227166,<$585
Huang High School,District,2917,"$1,910,635.00",$655.00,76.629414,81.182722,65.683922,81.316421,53.513884,$645-680
Johnson High School,District,4761,"$3,094,650.00",$650.00,77.072464,80.966394,66.057551,81.222432,53.539172,$645-680
Pena High School,Charter,962,"$585,858.00",$609.00,83.839917,84.044699,94.594595,95.945946,90.540541,$585-630


In [147]:
spending_math_scores = school_spending_df.groupby(["Spending Ranges (Per Student)"])["Average Math Score"].mean()
spending_reading_scores = school_spending_df.groupby(["Spending Ranges (Per Student)"])["Average Reading Score"].mean()
spending_passing_math = school_spending_df.groupby(["Spending Ranges (Per Student)"])["% Passing Math"].mean()
spending_passing_reading = school_spending_df.groupby(["Spending Ranges (Per Student)"])["% Passing Reading"].mean()
overall_passing_spending = school_spending_df.groupby(["Spending Ranges (Per Student)"])["% Overall Passing"].mean()

spending_summary = pd.DataFrame({"Average Math Score": spending_math_scores, "Average Reading Score": spending_reading_scores,
                                "% Passing Math": spending_passing_math, "% Passing Reading": spending_passing_reading,
                                "% Overall Passing": overall_passing_spending})

spending_summary

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Spending Ranges (Per Student),,,,,
<$585,83.455399,83.933814,93.460096,96.610877,90.369459
$585-630,81.899826,83.155286,87.133538,92.718205,81.418596
$630-645,78.518855,81.624473,73.484209,84.391793,62.857656
$645-680,76.997210,81.027843,66.164813,81.133951,53.526855


## Scores by School Size

In [148]:
size_bins = [0, 1000, 2000, 5000]

size_labels = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]

school_size_df = per_school_summary.copy()

school_size_df["School Size"] = pd.cut(school_size_df["Total Students"], size_bins, labels=size_labels, include_lowest=True)
school_size_df

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing,School Size
Bailey High School,District,4976,"$3,124,928.00",$628.00,77.048432,81.033963,66.680064,81.933280,54.642283,Large (2000-5000)
Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.061895,83.975780,94.133477,97.039828,91.334769,Medium (1000-2000)
Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.711767,81.158020,65.988471,80.739234,53.204476,Large (2000-5000)
Ford High School,District,2739,"$1,763,916.00",$644.00,77.102592,80.746258,68.309602,79.299014,54.289887,Large (2000-5000)
Griffin High School,Charter,1468,"$917,500.00",$625.00,83.351499,83.816757,93.392371,97.138965,90.599455,Medium (1000-2000)
Hernandez High School,District,4635,"$3,022,020.00",$652.00,77.289752,80.934412,66.752967,80.862999,53.527508,Large (2000-5000)
Holden High School,Charter,427,"$248,087.00",$581.00,83.803279,83.814988,92.505855,96.252927,89.227166,Small (<1000)
Huang High School,District,2917,"$1,910,635.00",$655.00,76.629414,81.182722,65.683922,81.316421,53.513884,Large (2000-5000)
Johnson High School,District,4761,"$3,094,650.00",$650.00,77.072464,80.966394,66.057551,81.222432,53.539172,Large (2000-5000)
Pena High School,Charter,962,"$585,858.00",$609.00,83.839917,84.044699,94.594595,95.945946,90.540541,Small (<1000)


In [149]:
size_math_scores = school_size_df.groupby(["School Size"])["Average Math Score"].mean()
size_reading_scores = school_size_df.groupby(["School Size"])["Average Reading Score"].mean()
size_passing_math = school_size_df.groupby(["School Size"])["% Passing Math"].mean()
size_passing_reading = school_size_df.groupby(["School Size"])["% Passing Reading"].mean()
overall_passing_size = school_size_df.groupby(["School Size"])["% Overall Passing"].mean()

size_summary = pd.DataFrame({"Average Math Score": size_math_scores, "Average Reading Score": size_reading_scores,
                            "% Passing Math": size_passing_math, "% Passing Reading": size_passing_reading,
                            "% Overall Passing": overall_passing_size})

size_summary

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School Size,,,,,
Small (<1000),83.821598,83.929843,93.550225,96.099437,89.883853
Medium (1000-2000),83.374684,83.864438,93.599695,96.790680,90.621535
Large (2000-5000),77.746417,81.344493,69.963361,82.766634,58.286003


## Scores by School Type

In [150]:
type_math_scores = per_school_summary.groupby(["School Type"])["Average Math Score"].mean()
type_reading_scores = per_school_summary.groupby(["School Type"])["Average Reading Score"].mean()
type_passing_math = per_school_summary.groupby(["School Type"])["% Passing Math"].mean()
type_passing_reading = per_school_summary.groupby(["School Type"])["% Passing Reading"].mean()
overall_passing_type = per_school_summary.groupby(["School Type"])["% Overall Passing"].mean()

type_summary = pd.DataFrame({"Average Math Score": type_math_scores, "Average Reading Score": type_reading_scores,
                            "% Passing Math": type_passing_math, "% Passing Reading": type_passing_reading,
                            "% Overall Passing": overall_passing_type})

type_summary

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School Type,,,,,
Charter,83.473852,83.896421,93.620830,96.586489,90.432244
District,76.956733,80.966636,66.548453,80.799062,53.672208
